In [12]:
import os
import string
from collections import defaultdict

import networkx as nx
import nltk
from nltk.util import ngrams

In [2]:
g_nlg = nx.read_gexf('out/elbulli_nlg.gexf')

In [3]:
g_dat = nx.read_gexf('out/elbulli_dat.gexf')

In [4]:
prep_prod_rels = ['bañado', 'alcohol', 'chocolate', 'lacteo', 'nuevaPasta', 'producto', 'relleno']
prep_ingr_rels = ['composicion']
prep_flav_rels = ['sabor']
prep_tech_rels = ['tecnica']

def get_prep_products(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_prod_rels}

def get_prep_ingredients(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_ingr_rels}

def get_prep_flavors(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_flav_rels}

def get_prep_components(g, prep):
    prods = get_prep_products(g, prep)
    ingrs = get_prep_ingredients(g, prep)
    flavs = get_prep_flavors(g, prep)
    return prods.union(ingrs).union(flavs)

def get_prep_techniques(g, prep):
    return {k for k in g[prep] if g[prep][k]['edgetype'] in prep_tech_rels}

def get_recip_preparations(g, recip):
    return [k for k in g[recip] if g[recip][k]['edgetype'] == 'elaboracion']

def get_recip_products(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_products(g, prep)}

def get_recip_ingredients(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_ingredients(g, prep)}

def get_recip_flavors(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_flavors(g, prep)}

def get_recip_components(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_components(g, prep)}

def get_recip_techniques(g, recip):
    preps = get_recip_preparations(g, recip)
    return {k for prep in preps for k in get_prep_techniques(g, prep)}

In [ ]:
def get_nlg_preparations(n):
    m = n.split('-')[0]
    return get_recip_preparations(g_nlg, m)

def clean_producto_sabor(i):
    return i.replace('Producto:', '').replace('sabor:', '')

In [ ]:
preps_dat_nlg_dict = defaultdict(list)
for n, data in g_dat.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        

In [5]:
ddd={'a':0,'b':0,'c':0, 'd':0, 'e':0, 'f':0,     'w':0, 'x':0, 'y':0, 'z':0}


def equals_preparations(n1, n2):
    ddd['a']+=1
    r = False
    data1 = g_dat.node[n1]
    data2 = g_dat.node[n2]
    if data1['title'] == data2['title']:
        ddd['b']+=1
        comps1 = get_prep_components(g_dat, n1)
        comps2 = get_prep_components(g_dat, n2)
        if comps1 == comps2:
            r = True
            ddd['c']+=1
        elif data1['title'] == 'Otros':
            ddd['d']+=1
        else:
            ddd['e']+=1
            
            nlg_preps1 = get_nlg_preparations(n1)
            for nlg_prep1 in nlg_preps1:
                nlg_comps1 = get_prep_components(g_nlg, nlg_prep1)
                nlg_comps1 = set(map(clean_producto_sabor, nlg_comps1))
                if nlg_comps1 == comps1:
                    ddd['w']+=1
                if nlg_comps1 == comps2:
                    ddd['x']+=1
            
            nlg_preps2 = get_nlg_preparations(n2)
            for nlg_prep2 in nlg_preps2:
                nlg_comps2 = get_prep_components(g_nlg, nlg_prep2)
                nlg_comps2 = set(map(clean_producto_sabor, nlg_comps2))
                if nlg_comps2 == comps1:
                    ddd['y']+=1
                if nlg_comps2 == comps2:
                    ddd['z']+=1
            
#             inters=comps1.intersection(comps2)
#             print(1, comps1)
#             print(2, comps2)
#             print('INTERSECT:', inters)
#             print('|INTERSECT|:', len(inters))
#             diff=comps1.symmetric_difference(comps2)
#             print('DIFF:', diff)
#             print('|DIFF|:', len(diff))
#             print()
    else:
        ddd['f']+=1
    return r

In [11]:
ddd

{'a': 49723652,
 'b': 909082,
 'c': 29426,
 'd': 873454,
 'e': 6202,
 'f': 48814570,
 'w': 287,
 'x': 116,
 'y': 116,
 'z': 287}

In [6]:
ddd

{'a': 0,
 'b': 0,
 'c': 0,
 'd': 0,
 'e': 0,
 'f': 0,
 'w': 0,
 'x': 0,
 'y': 0,
 'z': 0}

In [7]:
for n, data in g_dat.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        if data['title'] == 'Otros':
            print(n, data['title'])
            break

205-6 Otros


In [8]:
%%time

preps = set()
for n1, data1 in g_dat.nodes(data=True):
    if data1['nodetype'] == 'Elaboracion':
        for n2, data2 in g_dat.nodes(data=True):
            if data2['nodetype'] == 'Elaboracion':
                if n1 != n2:
                    if equals_preparations(n1, n2):
                        preps.add(tuple(sorted([n1, n2])))

CPU times: user 2min, sys: 104 ms, total: 2min
Wall time: 2min


In [9]:
assert(ddd['a']==ddd['b']+ddd['f'])

In [10]:
assert(ddd['b']==ddd['c']+ddd['d']+ddd['e'])

In [11]:
ddd

{'a': 49723652,
 'b': 909082,
 'c': 29426,
 'd': 873454,
 'e': 6202,
 'f': 48814570,
 'w': 287,
 'x': 116,
 'y': 116,
 'z': 287}

In [99]:
ddd

{'a': 49723652,
 'b': 907134,
 'c': 29388,
 'd': 871580,
 'e': 6166,
 'f': 48816518,
 'w': 287,
 'x': 116,
 'y': 116,
 'z': 287}

In [99]:
ddd

{'a': 49723652,
 'b': 907134,
 'c': 29388,
 'd': 871580,
 'e': 6166,
 'f': 48816518,
 'w': 287,
 'x': 116,
 'y': 116,
 'z': 287}

In [99]:
ddd

{'a': 49723652,
 'b': 907134,
 'c': 29388,
 'd': 871580,
 'e': 6166,
 'f': 48816518,
 'w': 287,
 'x': 116,
 'y': 116,
 'z': 287}

In [99]:
ddd

{'a': 49723652,
 'b': 907134,
 'c': 29388,
 'd': 871580,
 'e': 6166,
 'f': 48816518,
 'w': 287,
 'x': 116,
 'y': 116,
 'z': 287}

In [9]:
len(preps)

11298

In [20]:
# es_test = [
#     (1,2),(1,3),(2,3),(3,4),
#     (5,6),(6,7),(7,8),(7,9),
#     (10,11),(11,12),(12,13),
# #     (1,13),
#     (6,12),(1, 14)
# ]

In [69]:
%%time

ls=[]
for e1, e2 in es:
    found = False
    first_set = None
    to_remove=[]
    for x in ls:
        if (e1 in x or e2 in x) and not found:
            x.add(e1)
            x.add(e2)
            found = True
            first_set = x
        if x != first_set and (e1 in x or e2 in x) and found:
            x.add(e1)
            x.add(e2)
            for y in x:
                first_set.add(y)
            to_remove.append(x)
    if not found:
        ls.append({e1, e2})
    for y in to_remove:
        ls.remove(y)

In [70]:
len(ls)

821

In [71]:
for x in ls:
    ts = set()
    ingrs = set()
    descs = set()
    for y in x:
        ts.add(graph.node[y]['title'])
        ingrs.add(graph.node[y]['ingrs'])
        descs.add(graph.node[y]['desc'])
    if len(ts) != 1:
        print('AAA')
    break
#     if len(ingrs) != 1:
#         print('BBB')
#     if len(descs) != 1:
#         print('CCC')

In [72]:
list(x)[:4]

['799-8', '1003-5', '999-6', '262-9']

In [73]:
for y in list(ls[0])[:4]:
    print(graph.node[y]['title'])
    print(graph.node[y]['ingrs'])
    print(graph.node[y]['desc'])

Otros
          sal Maldon#          cacao en un espolvoreador#          la piel rallada de una mandarina#          la piel rallada de una lima#          la piel rallada de una naranja

Otros
     10 g de vinagre balsámico tradicional #          introducido en pipetas

Otros
   120 hojas de albahaca fresca de 0,5 cm #          de longitud#   100 g de azúcar

Otros
        4 brotes de romero fresco
######


In [74]:
equals_preparations('799-8', '1003-5')

True

In [75]:
get_prep_components(graph, '799-8')

set()

In [77]:
get_prep_products(graph, '799-8')

set()

In [81]:
{k for k in graph['799-8'] if graph['799-8'][k]['edgetype'] in prep_prod_rels}

set()

In [82]:
for k in graph['799-8']:
    print(k)

In [83]:
graph.node['799-8']

{'desc': '',
 'ingrs': '          sal Maldon#          cacao en un espolvoreador#          la piel rallada de una mandarina#          la piel rallada de una lima#          la piel rallada de una naranja',
 'label': '799-8',
 'nodetype': 'Elaboracion',
 'title': 'Otros'}

In [78]:
get_prep_ingredients(graph, '799-8')

set()

In [79]:
get_prep_flavors(graph, '799-8')

set()

In [76]:
get_prep_components(graph, '1003-5')

set()

In [68]:
c=d=0
for n, data in g_nlg.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        c+=1
for n, data in graph.nodes(data=True):
    if data['nodetype'] == 'Elaboracion':
        d+=1

In [69]:
c

4636

In [70]:
d

7052